# Transforming Data to Unlock Its Latent Value

In [ ]:
import os
import zipfile
import requests
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

path = 'data'

%matplotlib inline

## Download the Data

In [ ]:
def download_data(url, name, path='data'):
    if not os.path.exists(path):
        os.mkdir(path)

    response = requests.get(url)
    with open(os.path.join(path, name), 'wb') as f:
        f.write(response.content)
        
    z = zipfile.ZipFile(os.path.join(path, 'vehicles.zip'))
    z.extractall(path)

VEHICLES = 'http://www.fueleconomy.gov/feg/epadata/vehicles.csv.zip'

download_data(VEHICLES, 'vehicles.zip')

In [ ]:
vehicles = pd.read_csv(os.path.join(path, 'vehicles.csv'))

## Clean and Reorganize the Data

In [ ]:
select_columns = ['make', 'model', 'year', 'displ', 'cylinders', 'trany', 'drive', 'VClass','fuelType',  
                 'barrels08', 'city08', 'highway08', 'comb08', 'co2TailpipeGpm', 'fuelCost08']

vehicles = vehicles[select_columns][vehicles.year <= 2016].drop_duplicates().dropna()
vehicles = vehicles.sort_values(['make', 'model', 'year'])
vehicles.head()

## Create Category Aggregations

Hint: Look for object fields that have many categories. 

In [ ]:
def unique_col_values(df):
    for column in df:
        print(str(df[column].name) + " | " + str(len(df[column].unique())) + " | " + str(df[column].dtype))

unique_col_values(vehicles)

In [ ]:
#Create new trantype field that specifies whether the vehicle is Automatic or Manual. 
vehicles.loc[vehicles.trany.str[0] == 'A', 'trantype'] = 'Automatic'
vehicles.loc[vehicles.trany.str[0] == 'M', 'trantype'] = 'Manual'

#Create new model_type field that parses the model type from the model field. 
vehicles['model_type'] = vehicles.make + " " + vehicles.model.str.split().str.get(0)

#Create new category field that rolls up VClass into more general categories. 
small = ['Compact Cars','Subcompact Cars','Two Seaters','Minicompact Cars']
midsize = ['Midsize Cars']
large = ['Large Cars']

vehicles.loc[vehicles.VClass.isin(small), 'category'] = 'Small Cars'
vehicles.loc[vehicles.VClass.isin(midsize), 'category'] = 'Midsize Cars'
vehicles.loc[vehicles.VClass.isin(large), 'category'] = 'Large Cars'
vehicles.loc[vehicles.VClass.str.contains('Station'), 'category'] = 'Station Wagons'
vehicles.loc[vehicles.VClass.str.contains('Pickup'), 'category'] = 'Pickup Trucks'
vehicles.loc[vehicles.VClass.str.contains('Special Purpose'), 'category'] = 'Special Purpose'
vehicles.loc[vehicles.VClass.str.contains('Sport Utility'), 'category'] = 'Sport Utility'
vehicles.loc[(vehicles.VClass.str.contains('van')) | (vehicles.VClass.str.contains('van')),
               'category'] = 'Vans & Minivans'

#Create new fuel_category field that rolls up fuelType into more general categories. 
vehicles['fuel_category'] = ''
gas = ['Regular', 'Premium', 'Midgrade']
vehicles.loc[vehicles.fuelType.isin(gas), 'fuel_category'] = 'Gasoline'
vehicles.loc[vehicles.fuelType == 'Diesel', 'fuel_category'] = 'Diesel'
vehicles.loc[vehicles.fuel_category == '', 'fuel_category'] = 'Alternative/Hybrid'

## Create Categorical Fields from Continuous

In [ ]:
engine_categories = ['Very Small Engine', 'Small Engine','Moderate Engine', 
                     'Large Engine', 'Very Large Engine']
vehicles['engine_size'] = pd.qcut(vehicles.displ, 5, engine_categories)

efficiency_categories = ['Very Low Efficiency', 'Low Efficiency', 'Moderate Efficiency',
                        'High Efficiency', 'Very High Efficiency']
vehicles['fuel_efficiency'] = pd.qcut(vehicles.comb08, 5, efficiency_categories)

emmission_categories = ['Very Low Emmissions', 'Low Emmissions', 'Moderate Emmissions',
                       'High Emmissions', 'Very High Emmissions']
vehicles['emmission'] = pd.qcut(vehicles.co2TailpipeGpm, 5, emmission_categories)

fuelcost_categories = ['Very Low Fuel Cost', 'Low Fuel Cost', 'Moderate Fuel Cost',
                      'High Fuel Cost', 'Very High Fuel Cost']
vehicles['fuel_cost'] = pd.qcut(vehicles.fuelCost08, 5, fuelcost_categories)

## Cluster to Create Additional Categories

In [ ]:
vehicles_numeric = vehicles._get_numeric_data()
del vehicles_numeric['year']

vehicles_numeric_norm = vehicles_numeric.apply(lambda x: (x / x.max()))

In [ ]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=8)
clusters = model.fit_predict(vehicles_numeric_norm)
vehicles_numeric_norm['cluster'] = clusters

cluster_means = vehicles_numeric_norm.groupby(['cluster'], as_index=False).mean()
cluster_columns = ['displ','cylinders','barrels08','city08','highway08','comb08','co2TailpipeGpm','fuelCost08']

fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(cluster_means[cluster_columns], annot=True)

In [ ]:
model = KMeans(n_clusters=4)
clusters = model.fit_predict(vehicles_numeric_norm)
vehicles_numeric_norm['cluster'] = clusters

cluster_means = vehicles_numeric_norm.groupby(['cluster'], as_index=False).mean()
cluster_columns = ['displ','cylinders','barrels08','city08','highway08','comb08','co2TailpipeGpm','fuelCost08']

fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(cluster_means[cluster_columns], annot=True)

In [ ]:
vehicles['cluster'] = clusters
vehicles['cluster'][vehicles['cluster']==0] = 'Small Very Efficient'
vehicles['cluster'][vehicles['cluster']==1] = 'Large Inefficient'
vehicles['cluster'][vehicles['cluster']==2] = 'Midsized Balanced'
vehicles['cluster'][vehicles['cluster']==3] = 'Small Moderately Efficient'

## Aggregate and Filter

In [ ]:
def count_barchart(df, year, group_field, length, width):
    df_year = df[df.year == year]
    grouped = df_year.groupby(group_field, as_index=False).size().reset_index()
    grouped = grouped.rename(columns={0: 'count'}).sort_values('count', ascending=False)

    fig, ax = plt.subplots(figsize=(width,length))
    sns.barplot(x="count", y=group_field, data=grouped)

In [ ]:
count_barchart(vehicles, 2016, 'category', 6,8)

In [ ]:
count_barchart(vehicles, 1985, 'category', 6,8)

In [ ]:
count_barchart(vehicles, 2016, 'engine_size', 6, 8)

In [ ]:
count_barchart(vehicles, 2016, 'fuel_efficiency', 6, 8)

In [ ]:
count_barchart(vehicles, 2016, 'cluster', 6,8)

In [ ]:
count_barchart(vehicles, 2016, 'make', 12, 12)

## More Details with Pivoting

In [ ]:
def pivot_heatmap(df, year, rows, columns, values, width, length):
    df_year = df[df.year == year]
    df_pivot = df_year.pivot_table(values=values, index=rows, columns=columns, 
                                   aggfunc=np.size).dropna(axis=0, how='all')
    
    fig, ax = plt.subplots(figsize=(width,length))
    sns.heatmap(df_pivot, annot=True, fmt='g')

In [ ]:
pivot_heatmap(vehicles, 2016, 'fuel_efficiency','engine_size','comb08',15, 8)

In [ ]:
pivot_heatmap(vehicles, 1985, 'fuel_efficiency','engine_size','comb08',15, 8)

In [ ]:
pivot_heatmap(vehicles, 2016, 'cluster','category', 'comb08', 15, 10)

In [ ]:
pivot_heatmap(vehicles, 2016, ['engine_size', 'fuel_efficiency'],'category', 'comb08', 15, 15)

In [ ]:
pivot_heatmap(vehicles, 2016, 'make','category', 'comb08', 10, 10)

## Exploring Aggregations Over Time

In [ ]:
def multi_line(df, x, y):
    ax = df.groupby([x, y]).size().unstack(y).plot(figsize=(15,8), cmap="Set2")

In [ ]:
multi_line(vehicles, 'year', 'category')

In [ ]:
bmw = vehicles[vehicles.make == 'BMW']
multi_line(bmw, 'year', 'category')

In [ ]:
toyota = vehicles[vehicles.make == 'Toyota']
multi_line(toyota, 'year', 'category')

## Exploring Field Relationships

In [ ]:
def scatter_matrix(df, labels=None):
    ax = sns.pairplot(df, hue=labels, diag_kind='kde', size=2)
    plt.show()

scatter_matrix(vehicles_numeric_norm)

In [ ]:
scatter_matrix(vehicles_numeric_norm, labels="cluster")

In [ ]:
vehicles_numeric_norm['Cluster'] = vehicles['cluster']
sns.lmplot('displ', 'comb08', data=vehicles_numeric_norm, hue='Cluster', size=8, fit_reg=False)

In [ ]:
sns.lmplot('displ', 'fuelCost08', data=vehicles_numeric_norm, hue='Cluster', size=8, fit_reg=False)

## Exploring Entity Relationships (Graph Analysis)

In [ ]:
entity = 'make'
year = 2016

vehicles_year = vehicles[vehicles.year==year]

graph_year = pd.DataFrame(vehicles_year.groupby([entity,'cylinders','displ','trantype','drive',
                                                 'comb08','VClass', 'cluster'], 
                                                as_index=False).size()).reset_index()

graph_year = graph_year.rename(columns={0: 'count'})
graph_year['edge'] = (graph_year['cylinders'].map(str)
                      + graph_year['displ'].map(str)
                      + graph_year['trantype']
                      + graph_year['drive']
                      + graph_year['comb08'].map(str)
                      + graph_year['VClass']
                      + graph_year['cluster']
                     )

graph_year = graph_year[[entity, 'edge', 'count']]

In [ ]:
def df_to_graph(df, entity, edge):
    df2 = df.copy()
    graph_df = pd.merge(df, df2, how='inner', on=edge)
    graph_df = graph_df.groupby([entity + '_x', entity + '_y']).count().reset_index()
    graph_df = graph_df[graph_df[entity + '_x'] != graph_df[entity + '_y']]
    graph_df = graph_df[[entity + '_x', entity + '_y', edge]]
    return graph_df

In [ ]:
vehicle_make_graph = df_to_graph(graph_year, entity, 'edge')
vehicle_make_graph.head(10)

In [ ]:
import networkx as nx
import graph_tool.all as gt
import graph_tool as gt
from graph_tool import *

G = nx.from_pandas_dataframe(vehicle_make_graph, entity + '_x', entity + '_y', 'edge')

[Converting NetworkX to Graph-Tool](http://bbengfort.github.io/snippets/2016/06/23/graph-tool-from-networkx.html) by Benjamin Bengfort (converts NetworkX graphs to much prettier Graph-Tool graphs). 

In [ ]:
def get_prop_type(value, key=None):
    """
    Performs typing and value conversion for the graph_tool PropertyMap class.
    If a key is provided, it also ensures the key is in a format that can be
    used with the PropertyMap. Returns a tuple, (type name, value, key)
    """
    if isinstance(key, unicode):
        # Encode the key as ASCII
        key = key.encode('ascii', errors='replace')

    # Deal with the value
    if isinstance(value, bool):
        tname = 'bool'

    elif isinstance(value, int):
        tname = 'float'
        value = float(value)

    elif isinstance(value, float):
        tname = 'float'

    elif isinstance(value, unicode):
        tname = 'string'
        value = value.encode('ascii', errors='replace')

    elif isinstance(value, dict):
        tname = 'object'

    else:
        tname = 'string'
        value = str(value)

    return tname, value, key


def nx2gt(nxG):
    """
    Converts a networkx graph to a graph-tool graph.
    """
    # Phase 0: Create a directed or undirected graph-tool Graph
    gtG = gt.Graph(directed=nxG.is_directed())

    # Add the Graph properties as "internal properties"
    for key, value in nxG.graph.items():
        # Convert the value and key into a type for graph-tool
        tname, value, key = get_prop_type(value, key)

        prop = gtG.new_graph_property(tname) # Create the PropertyMap
        gtG.graph_properties[key] = prop     # Set the PropertyMap
        gtG.graph_properties[key] = value    # Set the actual value

    # Phase 1: Add the vertex and edge property maps
    # Go through all nodes and edges and add seen properties
    # Add the node properties first
    nprops = set() # cache keys to only add properties once
    for node, data in nxG.nodes_iter(data=True):

        # Go through all the properties if not seen and add them.
        for key, val in data.items():
            if key in nprops: continue # Skip properties already added

            # Convert the value and key into a type for graph-tool
            tname, _, key  = get_prop_type(val, key)

            prop = gtG.new_vertex_property(tname) # Create the PropertyMap
            gtG.vertex_properties[key] = prop     # Set the PropertyMap

            # Add the key to the already seen properties
            nprops.add(key)

    # Also add the node id: in NetworkX a node can be any hashable type, but
    # in graph-tool node are defined as indices. So we capture any strings
    # in a special PropertyMap called 'id' -- modify as needed!
    gtG.vertex_properties['id'] = gtG.new_vertex_property('string')

    # Add the edge properties second
    eprops = set() # cache keys to only add properties once
    for src, dst, data in nxG.edges_iter(data=True):

        # Go through all the edge properties if not seen and add them.
        for key, val in data.items():
            if key in eprops: continue # Skip properties already added

            # Convert the value and key into a type for graph-tool
            tname, _, key = get_prop_type(val, key)

            prop = gtG.new_edge_property(tname) # Create the PropertyMap
            gtG.edge_properties[key] = prop     # Set the PropertyMap

            # Add the key to the already seen properties
            eprops.add(key)

    # Phase 2: Actually add all the nodes and vertices with their properties
    # Add the nodes
    vertices = {} # vertex mapping for tracking edges later
    for node, data in nxG.nodes_iter(data=True):

        # Create the vertex and annotate for our edges later
        v = gtG.add_vertex()
        vertices[node] = v

        # Set the vertex properties, not forgetting the id property
        data['id'] = str(node)
        for key, value in data.items():
            gtG.vp[key][v] = value # vp is short for vertex_properties

    # Add the edges
    for src, dst, data in nxG.edges_iter(data=True):

        # Look up the vertex structs from our vertices mapping and add edge.
        e = gtG.add_edge(vertices[src], vertices[dst])

        # Add the edge properties
        for key, value in data.items():
            gtG.ep[key][e] = value # ep is short for edge_properties

    # Done, finally!
    return gtG


if __name__ == '__main__':

    # Create the networkx graph
    nxG = nx.Graph(name="Undirected Graph")
    nxG.add_node("v1", name="alpha", color="red")
    nxG.add_node("v2", name="bravo", color="blue")
    nxG.add_node("v3", name="charlie", color="blue")
    nxG.add_node("v4", name="hub", color="purple")
    nxG.add_node("v5", name="delta", color="red")
    nxG.add_node("v6", name="echo", color="red")

    nxG.add_edge("v1", "v2", weight=0.5, label="follows")
    nxG.add_edge("v1", "v3", weight=0.25, label="follows")
    nxG.add_edge("v2", "v4", weight=0.05, label="follows")
    nxG.add_edge("v3", "v4", weight=0.35, label="follows")
    nxG.add_edge("v5", "v4", weight=0.65, label="follows")
    nxG.add_edge("v6", "v4", weight=0.53, label="follows")
    nxG.add_edge("v5", "v6", weight=0.21, label="follows")

    for item in nxG.edges_iter(data=True):
        print(item)

    # Convert to graph-tool graph
    gtG = nx2gt(nxG)
    gtG.list_properties()

In [ ]:
def plot_graph(graph, width, length):
    g = nx2gt(graph)
    vlabel = g.vp['id']
    gt.graph_draw(g, output_size=(width,length), vertex_text=vlabel, vertex_font_weight=0.2, 
               vertex_size=5, vertex_fill_color='cyan')

plot_graph(G, 1200, 800)

In [ ]:
ego = nx.ego_graph(G, 'Nissan', 1)
plot_graph(ego, 500, 500)

In [ ]:
import community

def detect_communities(graph):
    partition = community.best_partition(graph)
    nx.set_node_attributes(graph, 'partition', partition)
    return graph, partition

make_communities = pd.DataFrame(detect_communities(G)[1].items(), 
                                columns=['make', 'community']).sort_values('community', ascending=True)

make_communities.head()

In [ ]:
import random
from copy import copy

##########################################################################
## Color Palettes
##########################################################################

FLATUI = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
PAIRED = [
    "#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99", "#e31a1c",
    "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a", "#ffff99", "#b15928",
]
SET1   = [
    "#e41a1c", "#377eb8", "#4daf4a",
    "#984ea3", "#ff7f00", "#ffff33",
    "#a65628", "#f781bf", "#999999"
]

PALETTES = {
    'flatui': FLATUI,
    'paired': PAIRED,
    'set1': SET1,
}

##########################################################################
## Color Utilities
##########################################################################

class ColorMap(object):
    """
    A helper for mapping categorical values to colors on demand.
    """

    def __init__(self, colors='flatui', shuffle=False):
        """
        Specify either a list of colors or one of the color names. If shuffle
        is True then the colors will be shuffled randomly.
        """
        self.mapping = {}
        self.colors = colors

        if shuffle:
            random.shuffle(self._colors)

    @property
    def colors(self):
        return self._colors

    @colors.setter
    def colors(self, value):
        """
        Converts color strings into a color listing.
        """
        if isinstance(value, basestring):
            if value not in PALETTES:
                raise ValueError("'{}' is not a registered color palette")
            self._colors = copy(PALETTES[value])
        elif isinstance(value, list):
            self._colors = value
        else:
            self._colors = list(value)

    def __call__(self, category):
        if category not in self.mapping:
            if self.colors:
                self.mapping[category] = self.colors.pop()
            else:
                raise ValueError(
                    "Not enough colors for this many categories!"
                )

        return self.mapping[category]

In [ ]:
def plot_community_graph(graph, community_df, width, length):
    g = nx2gt(G)
    vlabel = g.vp['id']
    vcolor = g.new_vertex_property('string') 
    vcmap = ColorMap('flatui', shuffle=False)
    for vertex in g.vertices():
        vcolor[vertex] = vcmap(community_df.community[vertex])
    gt.graph_draw(g, output_size=(width,length), vertex_text=vlabel, vertex_font_weight=0.2, 
               vertex_size=5, vertex_fill_color=vcolor)

plot_community_graph(G, make_communities, 1200, 800)

## Exploring Connections Over Time

In [ ]:
columns = ['make_x','make_y', 'edge','year']
graph_all_years = pd.DataFrame(columns=columns)

In [ ]:
for i in vehicles['year'].unique():
    vehicles_year = vehicles[vehicles.year==i]

    graph_year = pd.DataFrame(vehicles_year.groupby([entity,'cylinders','displ','trantype','drive',
                                                     'comb08','VClass', 'cluster'], 
                                                    as_index=False).size()).reset_index()

    graph_year = graph_year.rename(columns={0: 'count'})
    graph_year['edge'] = (graph_year['cylinders'].map(str)
                          + graph_year['displ'].map(str)
                          + graph_year['trantype']
                          + graph_year['drive']
                          + graph_year['comb08'].map(str)
                          + graph_year['VClass']
                          + graph_year['cluster']
                         )

    graph_year = graph_year[[entity, 'edge', 'count']]
    vehicle_make_graph = df_to_graph(graph_year, entity, 'edge')
    vehicle_make_graph['year'] = i
    graph_all_years = graph_all_years.append(vehicle_make_graph)

In [ ]:
graph_summary = graph_all_years.groupby(['make_x', 'year'], 
                                        as_index=False).sum()

graph_summary.head()

In [ ]:
def graph_multi_line(df, x, y):
    ax = df.groupby([x, y]).sum().unstack(y).plot(figsize=(15,8), cmap="jet")
    ax.legend(loc='center', bbox_to_anchor=(0.5, -0.35),
          ncol=5, fancybox=True, shadow=True, labels=df[y].unique())

graph_multi_line(graph_summary, 'year', 'make_x')

In [ ]:
makes = ['Chevrolet', 'Ford', 'Toyota', 'Honda', 'Nissan']

def graph_multi_line_makes(df, x, y):
    ax = df.groupby([x, y]).sum().unstack(y).plot(figsize=(15,8), cmap="jet")
    ax.legend(loc='center', bbox_to_anchor=(0.5, -0.15),
          ncol=5, fancybox=True, shadow=True, labels=df[y].unique())

graph_summary_makes = graph_summary[graph_summary.make_x.isin(makes)]
graph_multi_line_makes(graph_summary_makes, 'year', 'make_x')